In [1]:
%matplotlib inline
import netCDF4
import rasterio
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# local libraries
from coupling_PCR_FM.model_functions_v2 import PCR_model, CMF_model, DFM_model
from coupling_PCR_FM.utils import config_to_dict
%config Application.log_level="INFO"

In [3]:
# IMPORT MODEL SETTINGS FROM INI-FILE
argv1 = r'default_rename.set'
argv2 = r'paths.env'

# parse set/ini-file with central/general settings for coupling framework
config = config_to_dict(argv1)
# parse env-file for user-specific paths and environmental variables
envs = config_to_dict(argv2)
# combine
config.update(envs)
options = config
# parse dates
start_date = datetime.strptime(options['numerical_settings']['startTime'], '%Y-%m-%d')
end_date = datetime.strptime(options['numerical_settings']['endTime'], '%Y-%m-%d')

In [4]:
# setup output dir
cwd = os.getcwd() # note: this get changed by pcr initialization later on
out_dir = options['PCRpaths']['outputDirectoryPCR']
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
print(out_dir)
print(cwd)

/home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/
/home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim


## create PCR model BMI object

In [5]:
PCR_config_fn = os.path.join(cwd, options['hydrologic_model']['config_dir'], options['hydrologic_model']['config_file'])
PCR_in_dir = options['PCRpaths']['inputDirectoryPCR']
PCR_out_dir = os.path.join(out_dir, 'PCR')
print(PCR_in_dir)
print(PCR_config_fn)
print(PCR_out_dir)

PCR_bmi = PCR_model(PCR_config_fn, PCR_in_dir, PCR_out_dir,
                            start_date, end_date,)

/home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim/test_Elbe/PCR_Elbe/input30min/
/home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim/test_Elbe/PCR_Elbe/setup_PCR_30min_Elbe.ini
/home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/PCR


## create CMF model BMI object

In [6]:
CMF_engine = os.path.join(cwd, options['CMF_engine']['CMF_path'])
print(CMF_engine)
CMF_model_dir = os.path.join(cwd, options['routing_model']['model_dir'])
CMF_config_fn = os.path.join(CMF_model_dir, options['routing_model']['model_file'])
CMF_out_dir = os.path.join(out_dir, 'CMF')
     
CMF_bmi = CMF_model(CMF_engine, CMF_config_fn, CMF_model_dir, CMF_out_dir,
                         start_date, end_date, dt=86400)


2018-02-28 12:14:04,491 - INFO - Loading library from path /home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim/cama-flood_bmi/src/libcama.so


/home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim/cama-flood_bmi/src/libcama.so


## create DFM model BMI object

In [7]:
DFM_engine = os.path.join(cwd, options['DFM_engine']['DFM_path'])
DFM_model_dir = os.path.join(cwd, options['hydrodynamic_model']['model_dir'])
DFM_config_fn = os.path.join(DFM_model_dir, options['hydrodynamic_model']['model_file'])
DFM_out_dir = os.path.join(out_dir, 'DFM')
print(DFM_engine)
print(DFM_model_dir)
print(DFM_config_fn)
print(DFM_out_dir)


DFM_bmi = DFM_model(DFM_engine, DFM_config_fn, DFM_model_dir, DFM_out_dir,
                              start_date, end_date, dt=86400.)

2018-02-28 12:14:04,592 - INFO - Loading library from path /home/jannis/Programmes/DFLOWFM/lib/libdflowfm.so


/home/jannis/Programmes/DFLOWFM/lib/libdflowfm.so
/home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim/test_Elbe/DFM_Elbe/
/home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim/test_Elbe/DFM_Elbe/Elbe_1way_1d2dFM_400m_3200m.mdu
/home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/DFM


## intitialize DFM model

In [8]:
# initialize DFM first to expose model coordinates via BMI
DFM_bmi.initialize()

2018-02-28 12:14:04,722 - INFO - Ini file for DFM written to /home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/DFM/Elbe_1way_1d2dFM_400m_3200m.mdu
2018-02-28 12:14:04,723 - INFO - Loading model /home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/DFM/Elbe_1way_1d2dFM_400m_3200m.mdu in directory /home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/DFM
2018-02-28 12:14:27,019 - INFO - DFM initialized


## couple grids based on CMF inpmat file

In [9]:
# couple grids
PCR_bmi.couple_grid_to_grid(CMF_bmi)

2018-02-28 12:14:27,024 - INFO - Getting PCR model grid parameters.
2018-02-28 12:14:27,028 - INFO - Getting CMF model grid parameters.
2018-02-28 12:14:27,030 - INFO - Coupling PCR grid to CMF grid.
2018-02-28 12:14:27,030 - INFO - ./generate_inpmat 0.5 7.0 17.0 55.0 48.0 NtoS
2018-02-28 12:14:27,131 - INFO - Editing PCR ldd grid to deactivate routing in coupled cells.


## couple grids based on 1d DFM indices

Couple external 1d coordinates to internal model 2d grid. 

The model **ldd grid is also deactivated** at coupled cells.

**Note** that this should be excecuted before the CMF model is initalized as the lddl grid modification is done offline!

In [10]:
# get coupled dictionaries
CMF_bmi.couple_grid_to_1d(DFM_bmi)

2018-02-28 12:14:27,138 - INFO - Getting DFM model coordinates.
2018-02-28 12:14:27,171 - INFO - Coupling CMF grid to DFM 1D nodes.
2018-02-28 12:14:27,174 - INFO - Getting CMF model indices for xy coordinates.
2018-02-28 12:14:27,245 - INFO - Getting fraction of coupled 1d nodes based on area.
2018-02-28 12:14:27,257 - INFO - Editing CMF nextxy grid to deactivate routing in coupled cells.


## initialize CMF and PCR models

In [11]:
# initialize CMF model after nextxy had been adapted
CMF_bmi.initialize()
# initialize model after ldd has been modified
PCR_bmi.initialize()

2018-02-28 12:14:27,282 - INFO - removing tmp file diminfo_tmp.txt from model data dir
2018-02-28 12:14:27,283 - INFO - removing tmp file nextxy_tmp.bin from model data dir
2018-02-28 12:14:27,290 - INFO - removing tmp file inpmat-tmp.txt from model data dir
2018-02-28 12:14:27,291 - INFO - removing tmp file inpmat-tmp.bin from model data dir
2018-02-28 12:14:27,294 - INFO - Ini file for CMF written to /home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/CMF/input_flood.nam
2018-02-28 12:14:27,294 - INFO - Loading model /home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/CMF/input_flood.nam in directory /home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/CMF
2018-02-28 12:14:27,297 - INFO - CMF initialized
2018-02-28 12:14:27,302 - INFO - Ini file for PCR written to /home/jannis/PhD/code/GLOFRIMplusCAMA/output30min/PCR/setup_PCR_30min_Elbe.ini
2018-02-28 12:14:27,309 - INFO - Model run started at 2018-02-28 12:14:27.302970
2018-02-28 12:14:27,309 pcrglobwb_bmi_v203.configuration INFO Model ru

## define exchange of values in update states function

In [12]:
# determine area fraction for volume distribution
DFMidx = DFM_bmi.coupled_idx
CMFidx = CMF_bmi.coupled_idx
area_all = np.copy(DFM_bmi.get_var('ba'))
area_1d = area_all[DFMidx]

def calc_PCR_volume(runoff):
    cellarea = PCR_bmi.get_var('cellArea')
    cellarea = np.where(np.isnan(cellarea), 0, cellarea)
    PCR_volume = runoff * cellarea
    tot_PCR_volume = np.sum(PCR_volume)
    print 'sum runoff PCR * cellarea PCR: %.3e' % tot_PCR_volume
    return 

def set_CMF_runoff():
    """"
    coupling runoff between CMFan PCR model
    """
    runoff = np.copy(PCR_bmi.get_var('landSurfaceRunoff'))
    runoff = np.where(np.isnan(runoff), 0, runoff)
    calc_PCR_volume(runoff)
    CMF_bmi.set_var("runoff", runoff)
    
def set_DFM_discharge_and_update(updateStep):
    '''
    as the routing is turned off for coupled cells, only local discharge (=runoff) is passed on for coupled cells without an 
    upstream cell
    '''
    
    V_DFM_t0 = np.copy(DFM_bmi.get_var('vol1'))
    print 'V in DFM before adding Q from CMF: %.3e' % np.sum(V_DFM_t0)
    
    discharge_all = np.copy(CMF_bmi.get_var('outflw'))
    discharge = discharge_all[CMFidx] * CMF_bmi.options['dt'] #[m3/dt]
    delta_water_depth_dfm = discharge * CMF_bmi.coupled_area_frac / area_1d # [m/dt]
    DFM_bmi.set_var_index('rain', DFMidx, delta_water_depth_dfm)
    
    tot_Q_CMF2DFM = np.sum(discharge)
    print 'added V from CFM: %.3e' % tot_Q_CMF2DFM
    
    DFM_bmi.update(updateStep)
    
    V_DFM_t1 = np.copy(DFM_bmi.get_var('vol1'))
    print 'V in DFM after adding Q and update: %.3e' % np.sum(V_DFM_t1)
    d_V_DFM = np.sum(V_DFM_t1) - np.sum(V_DFM_t0)
    print 'increase of V DFM: %.3e' % d_V_DFM
    

## run coupled CMF - DFM model
# why is discharge from CMF negative???

In [13]:
updateStep = 86400.

# run coupled model for 2 timesteps
for i in range(2):
    PCR_bmi.update(1)
    set_CMF_runoff()
    CMF_bmi.update()
    set_DFM_discharge_and_update(updateStep)

2018-02-28 12:14:42,546 - INFO - reading forcings for time 2000-01-01
2018-02-28 12:14:42,546 pcrglobwb_bmi_v203.pcrglobwb INFO reading forcings for time 2000-01-01
2018-02-28 12:14:42,842 - INFO - updating model to time 2000-01-01
2018-02-28 12:14:42,842 pcrglobwb_bmi_v203.pcrglobwb INFO updating model to time 2000-01-01
2018-02-28 12:14:45,135 - INFO - PCR -> start_time: 2000-01-01, current_time 2000-01-01, timestep 1
2018-02-28 12:14:45,135 coupling_PCR_FM.model_functions_v2 INFO PCR -> start_time: 2000-01-01, current_time 2000-01-01, timestep 1
2018-02-28 12:14:45,167 - INFO - CMF -> start_time: 2000-01-01 00:00:00, current_time 2000-01-02 00:00:00, timestep 86400.0
2018-02-28 12:14:45,167 coupling_PCR_FM.model_functions_v2 INFO CMF -> start_time: 2000-01-01 00:00:00, current_time 2000-01-02 00:00:00, timestep 86400.0


sum runoff PCR * cellarea PCR: 2.313e+07
V in DFM before adding Q from CMF: 4.680e+09
added V from CFM: -6.001e+09


2018-02-28 12:14:53,683 - INFO - DFM -> start_time: 0.0, current_time 86400.0, timestep 86400.0
2018-02-28 12:14:53,683 coupling_PCR_FM.model_functions_v2 INFO DFM -> start_time: 0.0, current_time 86400.0, timestep 86400.0
2018-02-28 12:14:53,718 - INFO - reading forcings for time 2000-01-02
2018-02-28 12:14:53,718 pcrglobwb_bmi_v203.pcrglobwb INFO reading forcings for time 2000-01-02


V in DFM after adding Q and update: 4.680e+09
increase of V DFM: 5.527e+05


2018-02-28 12:14:54,173 - INFO - updating model to time 2000-01-02
2018-02-28 12:14:54,173 pcrglobwb_bmi_v203.pcrglobwb INFO updating model to time 2000-01-02
2018-02-28 12:14:55,374 - INFO - PCR -> start_time: 2000-01-01, current_time 2000-01-02, timestep 2
2018-02-28 12:14:55,374 coupling_PCR_FM.model_functions_v2 INFO PCR -> start_time: 2000-01-01, current_time 2000-01-02, timestep 2
2018-02-28 12:14:55,483 - INFO - CMF -> start_time: 2000-01-01 00:00:00, current_time 2000-01-03 00:00:00, timestep 86400.0
2018-02-28 12:14:55,483 coupling_PCR_FM.model_functions_v2 INFO CMF -> start_time: 2000-01-01 00:00:00, current_time 2000-01-03 00:00:00, timestep 86400.0


sum runoff PCR * cellarea PCR: 3.024e+07
V in DFM before adding Q from CMF: 4.680e+09
added V from CFM: -3.299e+09


2018-02-28 12:14:58,868 - INFO - DFM -> start_time: 0.0, current_time 172800.0, timestep 86400.0
2018-02-28 12:14:58,868 coupling_PCR_FM.model_functions_v2 INFO DFM -> start_time: 0.0, current_time 172800.0, timestep 86400.0


V in DFM after adding Q and update: 4.684e+09
increase of V DFM: 3.313e+06


In [14]:
# no clue what q1 is and why it has much more entries than total cells in DFM model... ask Arthur?
q1 = np.copy(DFM_bmi.get_var('q1'))
print len(q1), len(area_all)

98312 53638


In [15]:
# there is a variable qtotal_1d2d but it returns a NULL pointer for me... different for you?
DFM_bmi.get_var('qtotal_1d2d')

2018-02-28 12:14:59,042 - INFO - NULL pointer returned
2018-02-28 12:14:59,042 bmi.wrapper INFO NULL pointer returned


array(None, dtype=object)

## Finalize models

In [16]:
PCR_bmi.finalize()
CMF_bmi.finalize()
DFM_bmi.finalize()


2018-02-28 12:14:59,156 - INFO - cd /home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim
2018-02-28 12:14:59,156 bmi.wrapper INFO cd /home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim
2018-02-28 12:14:59,160 - INFO - cd /home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim
2018-02-28 12:14:59,160 bmi.wrapper INFO cd /home/jannis/PhD/code/GLOFRIMplusCAMA/glofrim
